In [ ]:
import random
import numpy as np
from collections import defaultdict, deque
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, Tuple, List, Optional, Set
import time
import json
import os
from datetime import datetime

class MaxFlowTabuSearch:
    """
    Implementazione Tabu Search per il problema del flusso massimo con visualizzazione migliorata.
    """

    def __init__(self, graph: Dict[Tuple[int, int], float], source: int, sink: int,
                 tabu_tenure: int = 20, max_iterations: int = 20000):
        self.source = source
        self.sink = sink
        self.graph = graph

        # Mappatura nodi
        self.node_map, self.reverse_node_map = self._create_node_mapping()
        self.source = self.node_map[source]
        self.sink = self.node_map[sink]

        # Build mapped graph
        self.graph_mapped = {}
        for (u, v), cap in graph.items():
            self.graph_mapped[(self.node_map[u], self.node_map[v])] = cap

        # Parameters
        self.tabu_tenure = tabu_tenure
        self.max_iterations = max_iterations

        # Data structures
        self.edges = list(self.graph_mapped.keys())
        self.capacities = np.array([self.graph_mapped[e] for e in self.edges], dtype=np.float32)
        self.edge_to_idx = {(u, v): i for i, (u, v) in enumerate(self.edges)}

        # Flow arrays
        self.current_flow = np.zeros(len(self.edges), dtype=np.float32)
        self.best_flow = np.zeros(len(self.edges), dtype=np.float32)
        self.temp_flow = np.zeros(len(self.edges), dtype=np.float32)
        self.initial_flow = np.zeros(len(self.edges), dtype=np.float32)

        # Statistics
        self.best_value = 0.0
        self.convergence_history = []
        self.convergence_iterations = []  # Track exact iteration numbers
        self.evaluations = 0
        self.best_iteration = 0
        self.tabu_list = {}
        self.start_time = time.time()

        # Build adjacency lists
        self._build_adjacency_lists()

        # Calculate capacities
        self.source_out_capacity = sum(cap for (u, v), cap in self.graph_mapped.items() if u == self.source)
        self.sink_in_capacity = sum(cap for (u, v), cap in self.graph_mapped.items() if v == self.sink)

        # Calculate optimal max flow
        self.optimal_max_flow_value = self._run_edmonds_karp()
        print(f"Optimal max flow (EK): {self.optimal_max_flow_value:.2f}")

        # Initialize with intelligent solution
        self._init_with_partial_edmonds_karp()

    def _create_node_mapping(self) -> Tuple[Dict[int, int], Dict[int, int]]:
        """Create mapping from original node IDs to consecutive indices."""
        all_nodes = set()
        for u, v in self.graph.keys():
            all_nodes.add(u)
            all_nodes.add(v)

        node_list = []
        if self.source in all_nodes:
            node_list.append(self.source)
        if self.sink in all_nodes and self.sink != self.source:
            node_list.append(self.sink)

        for node in sorted(list(all_nodes)):
            if node not in node_list:
                node_list.append(node)

        node_map = {node: i for i, node in enumerate(node_list)}
        reverse_map = {i: node for i, node in enumerate(node_list)}

        return node_map, reverse_map

    def _build_adjacency_lists(self) -> None:
        """Build adjacency lists for efficient graph navigation."""
        self.outgoing = defaultdict(list)
        self.incoming = defaultdict(list)

        for i, (u, v) in enumerate(self.edges):
            self.outgoing[u].append((i, v))
            self.incoming[v].append((i, u))

    def _run_edmonds_karp(self) -> float:
        """Edmonds-Karp implementation for maximum flow (reference solution)."""
        flow = 0.0
        residual = defaultdict(dict)

        # Initialize residual network
        for (u, v), cap in self.graph_mapped.items():
            residual[u][v] = cap
            residual[v][u] = 0.0

        while True:
            # BFS to find augmenting path
            parent = {}
            visited = set()
            queue = deque([self.source])
            visited.add(self.source)
            found = False

            while queue and not found:
                u = queue.popleft()
                for v, cap in residual[u].items():
                    if cap > 1e-6 and v not in visited:
                        parent[v] = u
                        visited.add(v)
                        if v == self.sink:
                            found = True
                            break
                        queue.append(v)

            if not found:
                break

            # Find minimum residual capacity
            path_flow = float('inf')
            v = self.sink
            while v != self.source:
                u = parent[v]
                path_flow = min(path_flow, residual[u][v])
                v = u

            # Update flows and residual capacities
            v = self.sink
            while v != self.source:
                u = parent[v]
                residual[u][v] -= path_flow
                residual[v][u] += path_flow
                v = u

            flow += path_flow

        return flow

    def _init_with_partial_edmonds_karp(self) -> None:
        """Initialize with a partial solution based on Edmonds-Karp."""
        #print("Generating intelligent initial solution...")
        optimal_flow = self._run_internal_edmonds_karp()
        factor = random.uniform(0.7, 0.95)  # Changed from original
        self.current_flow = optimal_flow * factor
        np.copyto(self.initial_flow, self.current_flow)
        self.best_value = self._calculate_flow_value(self.current_flow)
        np.copyto(self.best_flow, self.current_flow)
        print(f"Initial solution: {self.best_value:.2f} (vs optimal {self.optimal_max_flow_value:.2f})")

        # Record initial state
        self.convergence_history.append(self.best_value)
        self.convergence_iterations.append(0)

    def _run_internal_edmonds_karp(self) -> np.array:
        """Run Edmonds-Karp internally and return flow assignment."""
        flow = np.zeros(len(self.edges), dtype=np.float32)
        residual = {}

        # Initialize residual network
        for idx, (u, v) in enumerate(self.edges):
            residual[(u, v)] = self.capacities[idx]
            residual[(v, u)] = 0.0

        while True:
            # BFS to find augmenting path
            queue = deque([self.source])
            parent = {}
            visited = {self.source}
            found_path = False

            while queue and not found_path:
                u = queue.popleft()
                for (x, y), cap in residual.items():
                    if x == u and cap > 1e-6 and y not in visited:
                        visited.add(y)
                        parent[y] = (u, (x, y))
                        if y == self.sink:
                            found_path = True
                            break
                        queue.append(y)

            if not found_path:
                break

            # Find minimum residual capacity
            path_flow = float('inf')
            v = self.sink
            while v != self.source:
                u, edge = parent[v]
                path_flow = min(path_flow, residual[edge])
                v = u

            # Update flow and residual network
            v = self.sink
            while v != self.source:
                u, edge = parent[v]
                idx = self.edge_to_idx.get(edge, -1)
                if idx >= 0:
                    flow[idx] += path_flow
                    residual[edge] -= path_flow
                    rev_edge = (edge[1], edge[0])
                    residual[rev_edge] += path_flow
                v = u

        return flow

    def _calculate_flow_value(self, flow: np.array) -> float:
        """Calculate total flow value exiting the source."""
        self.evaluations += 1
        return sum(flow[idx] for idx, (u, v) in enumerate(self.edges) if u == self.source)

    def _is_feasible(self, flow: np.array) -> bool:
        """Check if a flow assignment is feasible."""
        # Capacity constraints
        if np.any(flow < -1e-7) or np.any(flow > self.capacities + 1e-7):
            return False

        # Flow conservation
        inflow = defaultdict(float)
        outflow = defaultdict(float)

        for idx, (u, v) in enumerate(self.edges):
            inflow[v] += flow[idx]
            outflow[u] += flow[idx]

        for node in set(inflow.keys()).union(outflow.keys()):
            if node == self.source or node == self.sink:
                continue
            if abs(inflow[node] - outflow[node]) > 1e-5:
                return False

        return True

    def _check_optimality_condition(self, flow_value: float) -> bool:
        """Check if flow equals sum of source outgoing or sink incoming capacities."""
        return (abs(flow_value - self.source_out_capacity) < 1e-6 or
                abs(flow_value - self.sink_in_capacity) < 1e-6)

    def _find_augmenting_path(self, flow: np.array) -> Optional[List[Tuple[int, bool]]]:
        """Find an augmenting path in the residual network."""
        visited = set()
        parent = {}
        queue = deque([self.source])

        while queue:
            node = queue.popleft()

            if node == self.sink:
                # Reconstruct path with direction information
                path_info = []
                current = self.sink
                while current != self.source:
                    u, (edge_idx, is_forward) = parent[current]
                    path_info.append((edge_idx, is_forward))
                    current = u
                return list(reversed(path_info))

            if node in visited:
                continue
            visited.add(node)

            # Forward edges
            for edge_idx, target in self.outgoing[node]:
                residual = self.capacities[edge_idx] - flow[edge_idx]
                if residual > 1e-6 and target not in visited:
                    parent[target] = (node, (edge_idx, True))
                    queue.append(target)

            # Backward edges
            for edge_idx, source_node in self.incoming[node]:
                if flow[edge_idx] > 1e-6 and source_node not in visited:
                    parent[source_node] = (node, (edge_idx, False))
                    queue.append(source_node)

        return None

    def _apply_path_flow(self, current_flow: np.array,
                        path_info: List[Tuple[int, bool]], amount: float) -> np.array:
        """Apply flow along an augmenting path."""
        np.copyto(self.temp_flow, current_flow)
        for edge_idx, is_forward in path_info:
            if is_forward:
                self.temp_flow[edge_idx] += amount
            else:
                self.temp_flow[edge_idx] -= amount
        return self.temp_flow.copy()

    def _get_neighborhood_moves(self, current_flow: np.array, iteration: int) -> List[Tuple]:
        """Generate neighborhood moves for the current solution."""
        moves = []

        # Edge-based moves (sample more edges for larger graphs)
        sample_size = min(len(self.edges), max(300, len(self.edges) // 10))
        sampled_edges = random.sample(range(len(self.edges)), sample_size)

        for edge_idx in sampled_edges:
            current_f = current_flow[edge_idx]
            capacity = self.capacities[edge_idx]

            # Increment move
            if current_f < capacity - 1e-6:
                step_size = capacity * random.uniform(0.05, 0.25)
                step = min(capacity - current_f, step_size)
                if step > 1e-6:
                    np.copyto(self.temp_flow, current_flow)
                    self.temp_flow[edge_idx] += step
                    is_tabu = edge_idx in self.tabu_list and self.tabu_list[edge_idx] > iteration
                    moves.append((self.temp_flow.copy(), edge_idx, step, is_tabu))

            # Decrement move
            if current_f > 1e-6:
                step_size = capacity * random.uniform(0.05, 0.25)
                step = min(current_f, step_size)
                if step > 1e-6:
                    np.copyto(self.temp_flow, current_flow)
                    self.temp_flow[edge_idx] -= step
                    is_tabu = edge_idx in self.tabu_list and self.tabu_list[edge_idx] > iteration
                    moves.append((self.temp_flow.copy(), edge_idx, -step, is_tabu))

        # Augmenting path move
        path_info = self._find_augmenting_path(current_flow)
        if path_info:
            min_residual = float('inf')
            for edge_idx, is_forward in path_info:
                if is_forward:
                    min_residual = min(min_residual,
                                     self.capacities[edge_idx] - current_flow[edge_idx])
                else:
                    min_residual = min(min_residual, current_flow[edge_idx])

            if min_residual > 1e-6:
                amount = min(min_residual, min_residual * random.uniform(0.1, 0.9))
                new_flow = self._apply_path_flow(current_flow, path_info, amount)
                path_edges = tuple([info[0] for info in path_info])
                moves.append((new_flow, path_edges, amount, False))

        # Edge exchange moves
        if len(sampled_edges) >= 2:
            for _ in range(min(20, len(sampled_edges) // 2)):
                edge1, edge2 = random.sample(sampled_edges, 2)
                if (current_flow[edge1] > 1e-6 and
                    current_flow[edge2] < self.capacities[edge2] - 1e-6):
                    max_transfer = min(current_flow[edge1],
                                     self.capacities[edge2] - current_flow[edge2])
                    transfer = max_transfer * random.uniform(0.1, 0.5)
                    if transfer > 1e-6:
                        np.copyto(self.temp_flow, current_flow)
                        self.temp_flow[edge1] -= transfer
                        self.temp_flow[edge2] += transfer
                        moves.append((self.temp_flow.copy(), (edge1, edge2), transfer, False))

        return moves

    def _diversify_solution(self, current_flow: np.array) -> np.array:
        """Diversify the current solution when stagnation is detected."""
        print("Applying diversification...")
        reset_factor = random.uniform(0.5, 0.9)  # More aggressive reset
        noise_factor = random.uniform(0.2, 0.4)  # More noise

        new_flow = current_flow * (1 - reset_factor) + self.initial_flow * reset_factor

        num_edges_to_perturb = min(len(self.edges) // 5, 50)
        edges_to_perturb = random.sample(range(len(self.edges)), num_edges_to_perturb)

        for edge_idx in edges_to_perturb:
            capacity = self.capacities[edge_idx]
            if capacity > 0:
                noise = random.uniform(-noise_factor, noise_factor) * capacity
                new_flow[edge_idx] = max(0, min(capacity, new_flow[edge_idx] + noise))

        return new_flow

    def _adaptive_parameters(self, iteration: int, stagnation_counter: int) -> bool:
        """Adapt algorithm parameters during the search."""
        if stagnation_counter > 100:  # More sensitive to stagnation
            self.tabu_tenure = min(int(self.tabu_tenure * 1.25), 150)  # Larger changes
        elif stagnation_counter < 50:
            self.tabu_tenure = max(int(self.tabu_tenure * 0.9), 10)

        if iteration % 2000 == 0 and iteration > 0 and stagnation_counter > 300:
            self.current_flow = self._diversify_solution(self.current_flow)
            return True

        return False

    def plot_convergence(self, filename: Optional[str] = None, title_suffix: str = "") -> None:
        """Generate and optionally save convergence plot with accurate iteration tracking."""
        plt.figure(figsize=(12, 7))

        # Use exact iteration points
        iterations = np.array(self.convergence_iterations)
        values = np.array(self.convergence_history)

        # Create main plot
        plt.plot(iterations, values, 'b-', linewidth=2, label='Best Flow Value')

        # Optimal value line
        plt.axhline(y=self.optimal_max_flow_value, color='r', linestyle='--',
                   linewidth=1.5, label=f'Optimal (EK): {self.optimal_max_flow_value:.2f}')

        # Mark best solution point
        plt.scatter([self.best_iteration], [self.best_value], color='g',
                   s=100, zorder=5, label=f'Best: {self.best_value:.2f} at {self.best_iteration}')

        # Enhanced formatting
        plt.title(f'Tabu Search Convergence {title_suffix}\n'
                 f'Nodes: {len(self.node_map)}, Edges: {len(self.edges)}', pad=20)
        plt.xlabel('Iterations', labelpad=10)
        plt.ylabel('Flow Value', labelpad=10)
        plt.grid(True, alpha=0.3)
        plt.legend(loc='lower right', framealpha=1)

        # Adjust limits to highlight convergence
        y_min = min(values.min(), self.optimal_max_flow_value*0.9)
        y_max = max(values.max(), self.optimal_max_flow_value*1.05)
        plt.ylim(y_min, y_max)

        # Add info box
        elapsed_time = time.time() - self.start_time
        info_text = (f"Optimal: {self.optimal_max_flow_value:.2f}\n"
                    f"Reached at: {self.best_iteration} iter\n"
                    f"Evaluations: {self.evaluations}\n"
                    f"Time: {elapsed_time:.2f}s")
        plt.gcf().text(0.15, 0.7, info_text, bbox=dict(facecolor='white', alpha=0.8))

        if filename:
            plt.savefig(filename, dpi=300, bbox_inches='tight')
            plt.close()
        else:
            plt.show()

    def search(self, seed: Optional[int] = None) -> Tuple[float, np.array, int, int]:
        """Execute the improved Tabu Search algorithm with accurate tracking."""
        if seed is not None:
            random.seed(seed)
            np.random.seed(seed)

        #print(f"Starting search with {len(self.edges)} edges...")
        self.convergence_history = [self.best_value]
        self.convergence_iterations = [0]
        stagnation_counter = 0
        max_stagnation = 1500
        self.tabu_list = {}
        self.start_time = time.time()

        for iteration in range(self.max_iterations):
            diversified = self._adaptive_parameters(iteration, stagnation_counter)
            if diversified:
                stagnation_counter = 0
                print(f"Diversification applied at iteration {iteration}")

            moves = self._get_neighborhood_moves(self.current_flow, iteration)
            if not moves:
                print(f"No available moves at iteration {iteration}")
                break

            best_move_info = None
            best_value_candidate = -float('inf')
            best_flow_candidate = None
            aspiration_move = False

            for move_flow, move_info, delta, is_tabu in moves:
                if not self._is_feasible(move_flow):
                    continue

                value = self._calculate_flow_value(move_flow)

                if is_tabu and value > self.best_value:
                    best_move_info = move_info
                    best_value_candidate = value
                    best_flow_candidate = move_flow
                    aspiration_move = True
                    break

                if not is_tabu and value > best_value_candidate:
                    best_move_info = move_info
                    best_value_candidate = value
                    best_flow_candidate = move_flow
                    aspiration_move = False

            if best_flow_candidate is None:
                for move_flow, move_info, delta, is_tabu in moves:
                    if self._is_feasible(move_flow):
                        value = self._calculate_flow_value(move_flow)
                        if value > best_value_candidate:
                            best_move_info = move_info
                            best_value_candidate = value
                            best_flow_candidate = move_flow
                            break

                if best_flow_candidate is None:
                    stagnation_counter += 1
                    if stagnation_counter > max_stagnation:
                        print(f"Stopped due to stagnation at iteration {iteration}")
                        break
                    continue

            self.current_flow = best_flow_candidate
            current_value = self._calculate_flow_value(self.current_flow)

            if isinstance(best_move_info, int):
                self.tabu_list[best_move_info] = iteration + self.tabu_tenure
            elif isinstance(best_move_info, tuple):
                for idx in best_move_info:
                    self.tabu_list[idx] = iteration + self.tabu_tenure

            if iteration % 100 == 0:
                self.tabu_list = {k: v for k, v in self.tabu_list.items() if v > iteration}

            if current_value > self.best_value:
                self.best_value = current_value
                np.copyto(self.best_flow, self.current_flow)
                self.best_iteration = iteration
                stagnation_counter = 0
                #print(f"New best: {self.best_value:.2f} at iteration {iteration}")
                #if aspiration_move:
                #    print(" (found via aspiration)")
            else:
                stagnation_counter += 1

            # Record convergence every 50 iterations AND at every improvement
            if iteration % 50 == 0 or current_value > self.convergence_history[-1] or iteration == self.max_iterations - 1:
                self.convergence_history.append(self.best_value)
                self.convergence_iterations.append(iteration)

            if abs(self.best_value - self.optimal_max_flow_value) < 1e-6:
                print(f"🎯 Optimal solution {self.optimal_max_flow_value:.2f} reached at iteration {iteration}")
                break

            if self._check_optimality_condition(self.best_value):
                print(f"Optimality condition met at iteration {iteration}")
                break

            if iteration % 2000 == 0 and iteration > 0:
                print(f"Iteration {iteration}: Best = {self.best_value:.2f}, "
                      f"Stagnation = {stagnation_counter}, Tabu tenure = {self.tabu_tenure}")

        elapsed_time = time.time() - self.start_time
        print(f"Search completed after {iteration+1} iterations")
        print(f"Best solution: {self.best_value:.2f}")
        print(f"Objective function evaluations: {self.evaluations}")
        print(f"Elapsed time: {elapsed_time:.2f} seconds")

        # Ensure final point is recorded
        if self.convergence_iterations[-1] != iteration:
            self.convergence_history.append(self.best_value)
            self.convergence_iterations.append(iteration)

        return float(self.best_value), self.best_flow, int(self.best_iteration), int(self.evaluations)

def read_instance(filename: str) -> Tuple[Dict[Tuple[int, int], float], int, int, int, int]:
    """Read graph instance data from file."""
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f.readlines() if line.strip()]

    n_nodes = int(lines[0])
    n_edges = int(lines[1])
    source = int(lines[2])
    sink = int(lines[3])
    graph = {}

    for i in range(4, min(4 + n_edges, len(lines))):
        parts = lines[i].split()
        if len(parts) >= 3:
            u, v, capacity = int(parts[0]), int(parts[1]), float(parts[2])
            graph[(u, v)] = capacity

    return graph, source, sink, n_nodes, n_edges

def convert_numpy_types(obj):
    """Convert numpy types to native Python types for JSON serialization"""
    if isinstance(obj, np.generic):
        return obj.item()
    elif isinstance(obj, dict):
        return {k: convert_numpy_types(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(v) for v in obj]
    return obj

def plot_average_convergence(results: List[Dict], filename: str, optimal_value: float) -> None:
    """Plot average convergence across multiple runs with correct iteration mapping."""
    plt.figure(figsize=(12, 7))

    # Find maximum iteration count across all runs
    max_iter = max(len(r['convergence_iterations']) for r in results)

    # Prepare matrix for interpolation
    all_values = []
    for r in results:
        x_orig = np.array(r['convergence_iterations'])
        y_orig = np.array(r['convergence_history'])

        # Create interpolation function
        x_new = np.linspace(0, x_orig[-1], max_iter)
        y_interp = np.interp(x_new, x_orig, y_orig)
        all_values.append(y_interp)

    # Calculate statistics
    avg_convergence = np.mean(all_values, axis=0)
    std_convergence = np.std(all_values, axis=0)
    x_values = np.linspace(0, x_new[-1], max_iter)

    # Plot with shaded standard deviation
    plt.plot(x_values, avg_convergence, 'b-', linewidth=2, label='Average Flow')
    plt.fill_between(x_values, avg_convergence - std_convergence,
                    avg_convergence + std_convergence, alpha=0.2)

    # Optimal value line
    plt.axhline(y=optimal_value, color='r', linestyle='--',
               linewidth=1.5, label=f'Optimal (EK): {optimal_value:.2f}')

    # Find and mark convergence point
    convergence_idx = np.where(avg_convergence >= optimal_value - 1e-6)[0]
    if len(convergence_idx) > 0:
        conv_iter = int(x_values[convergence_idx[0]])
        plt.scatter([conv_iter], [optimal_value], color='g',
                   s=100, zorder=5, label=f'Convergence at {conv_iter}')

    plt.title(f'Average Convergence for {os.path.basename(filename)}\n'
             f'{len(results)} runs', pad=20)
    plt.xlabel('Iterations', labelpad=10)
    plt.ylabel('Flow Value', labelpad=10)
    plt.grid(True, alpha=0.3)
    plt.legend(loc='lower right')

    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()

def run_experiments(output_dir: str = "results") -> Dict[str, Dict]:
    """Run experiments for all found instances with accurate convergence tracking."""
    print("=== IMPROVED TABU SEARCH FOR MAXIMUM FLOW ===")
    print("Version with precise convergence tracking and visualization")

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    instance_files = glob.glob("network_*.txt")
    if not instance_files:
        print("No 'network_*.txt' instances found in current directory.")
        return {}

    results = {}

    for filename in sorted(instance_files):
        print(f"\n--- Processing instance: {filename} ---")
        try:
            graph, source, sink, n_nodes, n_edges = read_instance(filename)
            print(f"Graph: Nodes={n_nodes}, Edges={n_edges}")
            print(f"Source={source}, Sink={sink}")

            runs_results = []
            num_runs = 10

            for run in range(num_runs):
                print(f"\n--- Run {run + 1}/{num_runs} for {filename} ---")
                solver = MaxFlowTabuSearch(graph, source, sink,
                                         tabu_tenure=20, max_iterations=20000)
                seed = run + 42
                best_value, best_flow, best_iteration, evaluations = solver.search(seed=seed)

                runs_results.append({
                    'best_value': float(best_value),
                    'best_iteration': int(best_iteration),
                    'evaluations': int(evaluations),
                    'optimal_max_flow_value': float(solver.optimal_max_flow_value),
                    'convergence_history': [float(x) for x in solver.convergence_history],
                    'convergence_iterations': [int(x) for x in solver.convergence_iterations]
                })

                # Save individual run plot
                plot_path = os.path.join(output_dir, f"{os.path.basename(filename)}_run{run+1}_convergence.png")
                solver.plot_convergence(plot_path, title_suffix=f"({os.path.basename(filename)}, Run {run+1})")

                print(f"Run {run + 1} result: Flow={best_value:.2f}, Iter={best_iteration}, Eval={evaluations}")

            # Create average convergence plot
            avg_plot_path = os.path.join(output_dir, f"{os.path.basename(filename)}_average_convergence.png")
            plot_average_convergence(runs_results, avg_plot_path, runs_results[0]['optimal_max_flow_value'])

            # Calculate statistics
            best_values = [r['best_value'] for r in runs_results]
            best_iterations = [r['best_iteration'] for r in runs_results]
            evaluations_list = [r['evaluations'] for r in runs_results]
            optimal_value = runs_results[0]['optimal_max_flow_value']

            gaps = [(optimal_value - bv) / optimal_value * 100 if optimal_value > 0 else 0
                   for bv in best_values]

            result = {
                'best': float(max(best_values)),
                'mean': float(np.mean(best_values)),
                'std': float(np.std(best_values)),
                'avg_iterations': float(np.mean(best_iterations)),
                'avg_evaluations': float(np.mean(evaluations_list)),
                'optimal_value_ek': float(optimal_value),
                'avg_gap': float(np.mean(gaps)),
                'best_gap': float(min(gaps)),
                'n_nodes': int(n_nodes),
                'n_edges': int(n_edges),
                'success_rate': float(sum(1 for gap in gaps if gap < 0.01) / len(gaps) * 100),
                'convergence_plot': avg_plot_path
            }

            results[filename] = result

            # Save results
            instance_result_path = os.path.join(output_dir, f"{os.path.basename(filename)}_results.json")
            with open(instance_result_path, 'w') as f:
                json.dump(convert_numpy_types(result), f, indent=2)

            print(f"\n--- Final statistics for {filename} ---")
            print(f"Optimal (Edmonds-Karp): {result['optimal_value_ek']:.2f}")
            print(f"Best Flow (TS): {result['best']:.2f}")
            print(f"Mean Flow (TS): {result['mean']:.2f} ± {result['std']:.2f}")
            print(f"Standard Deviation: {result['std']:.2f}")
            #print(f"Average optimality gap: {result['avg_gap']:.2f}%")
            #print(f"Best gap: {result['best_gap']:.2f}%")
            #print(f"Success rate (<0.01%): {result['success_rate']:.1f}%")
            print(f"Average iterations: {result['avg_iterations']:.0f}")
            print(f"Average evaluations: {result['avg_evaluations']:.0f}")

        except Exception as e:
            print(f"Error processing {filename}: {e}")
            import traceback
            traceback.print_exc()

    return results

if __name__ == "__main__":
    experiment_results = run_experiments()

    print("\n=== EXPERIMENT SUMMARY ===")
    for filename, res in experiment_results.items():
        print(f"\n{os.path.basename(filename)}:")
        print(f"Nodes: {res['n_nodes']}, Edges: {res['n_edges']}")
        print(f"Optimal EK: {res['optimal_value_ek']:.2f}")
        print(f"Best TS: {res['best']:.2f} (gap: {res['best_gap']:.2f}%)")
        print(f"Mean TS: {res['mean']:.2f}±{res['std']:.2f}")
        print(f"Standard Deviation: {res['std']:.2f}")
        #print(f"Success: {res['success_rate']:.1f}%")
        #print(f"Evals: {res['avg_evaluations']:.0f}")
        print(f"Average iterations: {res['avg_iterations']:.0f}")
        print(f"Average evaluations: {res['avg_evaluations']:.0f}")

In [ ]:
import random
import numpy as np
from collections import deque, defaultdict
import time
import os
import matplotlib.pyplot as plt
from typing import Dict, Tuple, List, Optional
from multiprocessing import Pool, cpu_count
import json


class MaxFlowTabuSearchOptimized:
    def __init__(self, graph: Dict[Tuple[int, int], float], source: int, sink: int,
                 tabu_tenure=20, max_iterations=20000):
        self.source = source
        self.sink = sink
        self.graph = graph
        self.edges = list(graph.keys())
        self.capacities = np.array([graph[e] for e in self.edges], dtype=np.float32)
        self.edge_to_idx = {(u, v): i for i, (u, v) in enumerate(self.edges)}
        self.source_edges = [i for i, (u, v) in enumerate(self.edges) if u == self.source]
        self.n_edges = len(self.edges)
        self.tabu_tenure = tabu_tenure
        self.max_iterations = max_iterations
        self.current_flow = np.zeros(len(self.edges), dtype=np.float32)
        self.best_flow = np.zeros(len(self.edges), dtype=np.float32)
        self.best_value = 0.0
        self.convergence_history = []
        self.convergence_iterations = []
        self.evaluations = 0
        self.best_iteration = 0
        self.start_time = time.time()
        self._precompute_adjacency()
        self.optimal_max_flow_value = self._run_edmonds_karp()
        self.source_out_capacity = sum(cap for (u, v), cap in self.graph.items() if u == self.source)
        self.sink_in_capacity = sum(cap for (u, v), cap in self.graph.items() if v == self.sink)
        self._init_with_partial_edmonds_karp()

    def _precompute_adjacency(self):
        self.adj = defaultdict(set)
        for (u, v) in self.graph:
            self.adj[u].add(v)

    def _run_edmonds_karp(self):
        residual = defaultdict(float)
        for (u, v), cap in self.graph.items():
            residual[(u, v)] = cap
            residual[(v, u)] = 0.0
        flow = 0.0
        while True:
            parent = {}
            visited = {self.source}
            queue = deque([self.source])
            found = False
            while queue and not found:
                u = queue.popleft()
                for v in self.adj[u]:
                    if residual[(u, v)] > 1e-6 and v not in visited:
                        visited.add(v)
                        parent[v] = u
                        if v == self.sink:
                            found = True
                            break
                        queue.append(v)
            if not found:
                break
            path_flow = float('inf')
            v = self.sink
            while v != self.source:
                u = parent[v]
                path_flow = min(path_flow, residual[(u, v)])
                v = u
            v = self.sink
            while v != self.source:
                u = parent[v]
                residual[(u, v)] -= path_flow
                residual[(v, u)] += path_flow
                v = u
            flow += path_flow
        return flow

    def _init_with_partial_edmonds_karp(self):
        optimal_flow = np.zeros(len(self.edges), dtype=np.float32)
        residual = defaultdict(float)
        for idx, (u, v) in enumerate(self.edges):
            residual[(u, v)] = self.capacities[idx]
            residual[(v, u)] = 0.0
        while True:
            parent = {}
            visited = {self.source}
            queue = deque([self.source])
            found = False
            while queue and not found:
                u = queue.popleft()
                for v in self.adj[u]:
                    if residual[(u, v)] > 1e-6 and v not in visited:
                        visited.add(v)
                        parent[v] = u
                        if v == self.sink:
                            found = True
                            break
                        queue.append(v)
            if not found:
                break
            path_flow = float('inf')
            v = self.sink
            while v != self.source:
                u = parent[v]
                path_flow = min(path_flow, residual[(u, v)])
                v = u
            v = self.sink
            while v != self.source:
                u = parent[v]
                edge = (u, v)
                idx = self.edge_to_idx.get(edge, -1)
                if idx >= 0:
                    residual[(u, v)] -= path_flow
                    residual[(v, u)] += path_flow
                    optimal_flow[idx] += path_flow
                v = u
        factor = random.uniform(0.7, 0.95)
        self.current_flow[:] = optimal_flow * factor
        self.best_value = self._calculate_flow_value(self.current_flow)
        self.best_flow[:] = self.current_flow
        self.convergence_history = [self.best_value]
        self.convergence_iterations = [0]

    def _calculate_flow_value(self, flow):
        self.evaluations += 1
        return np.sum(flow[self.source_edges])

    def _get_neighborhood_moves(self, current_flow):
        moves = []
        sample_size = min(len(self.edges), max(50, len(self.edges) // 40))
        sampled_edges = random.sample(range(len(self.edges)), sample_size)
        for edge_idx in sampled_edges:
            curr = current_flow[edge_idx]
            cap = self.capacities[edge_idx]
            if curr < cap - 1e-6:
                step = min(cap - curr, cap * 0.2)
                new_flow = current_flow.copy()
                new_flow[edge_idx] += step
                is_tabu = edge_idx in self.tabu_dict and self.tabu_dict[edge_idx] > 0
                moves.append((new_flow, edge_idx, step, is_tabu))
            if curr > 1e-6:
                step = min(curr, cap * 0.2)
                new_flow = current_flow.copy()
                new_flow[edge_idx] -= step
                is_tabu = edge_idx in self.tabu_dict and self.tabu_dict[edge_idx] > 0
                moves.append((new_flow, edge_idx, -step, is_tabu))
        if self.n_edges <= 1000 and len(sampled_edges) >= 2:
            edge1, edge2 = random.sample(sampled_edges, 2)
            if current_flow[edge1] > 1e-6 and current_flow[edge2] < self.capacities[edge2] - 1e-6:
                transfer = min(current_flow[edge1], self.capacities[edge2] - current_flow[edge2]) * 0.3
                new_flow = current_flow.copy()
                new_flow[edge1] -= transfer
                new_flow[edge2] += transfer
                moves.append((new_flow, (edge1, edge2), transfer, False))
        return moves

    def _diversify_solution(self, current_flow):
        reset_factor = random.uniform(0.5, 0.8)
        noise_factor = random.uniform(0.1, 0.2)
        new_flow = current_flow * (1 - reset_factor)
        num_perturb = min(len(self.edges) // 10, 10)
        indices = random.sample(range(len(self.edges)), num_perturb)
        for idx in indices:
            cap = self.capacities[idx]
            noise = random.uniform(-noise_factor, noise_factor) * cap
            new_flow[idx] = max(0, min(cap, new_flow[idx] + noise))
        return new_flow

    def plot_convergence(self, filename=None, title_suffix=""):
        plt.figure(figsize=(10, 6))
        plt.plot(self.convergence_iterations, self.convergence_history, label='Best Flow', color='blue')
        plt.axhline(y=self.optimal_max_flow_value, color='red', linestyle='--', label='Optimal (Edmonds-Karp)')
        #plt.axhline(y=min(self.source_out_capacity, self.sink_in_capacity), 
        #           color='green', linestyle=':', label='Upper Bound')
        plt.title(f'Convergence {title_suffix}')
        plt.xlabel('Iterations')
        plt.ylabel('Flow Value')
        plt.grid(True)
        plt.legend()
        if filename:
            plt.savefig(filename, dpi=300, bbox_inches='tight')
            plt.close()
        else:
            plt.show()

    def search(self, seed=None):
        if seed is not None:
            random.seed(seed)
            np.random.seed(seed)
        stagnation_counter = 0
        max_stagnation = 1000
        self.tabu_dict = defaultdict(int)
        upper_bound = min(self.source_out_capacity, self.sink_in_capacity)
        for iteration in range(self.max_iterations):
            if stagnation_counter >= max_stagnation:
                self.current_flow = self._diversify_solution(self.current_flow)
                stagnation_counter = 0
            moves = self._get_neighborhood_moves(self.current_flow)
            if not moves:
                break
            best_move = None
            best_val = -np.inf
            for move_flow, idx, delta, is_tabu in moves:
                val = self._calculate_flow_value(move_flow)
                if (not is_tabu or val > self.best_value) and val > best_val:
                    best_val = val
                    best_move = (move_flow, idx)
            if best_move is None:
                stagnation_counter += 1
                continue
            new_flow, edge_idx = best_move
            self.current_flow = new_flow
            if isinstance(edge_idx, tuple):
                for e in edge_idx:
                    self.tabu_dict[e] = self.tabu_tenure
            else:
                self.tabu_dict[edge_idx] = self.tabu_tenure
            self.tabu_dict = {k: v-1 for k, v in self.tabu_dict.items() if v > 1}
            current_val = self._calculate_flow_value(self.current_flow)
            if current_val > self.best_value:
                self.best_value = current_val
                self.best_flow[:] = self.current_flow
                self.best_iteration = iteration
                stagnation_counter = 0
            else:
                stagnation_counter += 1
            if iteration % 100 == 0 or current_val > self.convergence_history[-1]:
                self.convergence_history.append(current_val)
                self.convergence_iterations.append(iteration)
            if abs(current_val - upper_bound) < 1e-6:
                break
        elapsed_time = time.time() - self.start_time
        return {
            'convergence_history': self.convergence_history,
            'convergence_iterations': self.convergence_iterations,
            'best_value': self.best_value,
            'best_flow': self.best_flow,
            'best_iteration': self.best_iteration,
            'evaluations': self.evaluations,
            'elapsed_time': elapsed_time,
            'source_out_capacity': self.source_out_capacity,
            'sink_in_capacity': self.sink_in_capacity,
            'optimal_flow': self.optimal_max_flow_value
        }


def read_instance(filename):
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f.readlines() if line.strip()]
    n_nodes = int(lines[0])
    n_edges = int(lines[1])
    source = int(lines[2])
    sink = int(lines[3])
    graph = {}
    for i in range(4, min(4 + n_edges, len(lines))):
        parts = lines[i].split()
        if len(parts) >= 3:
            u, v, capacity = int(parts[0]), int(parts[1]), float(parts[2])
            graph[(u, v)] = capacity
    return graph, source, sink, n_nodes, n_edges


def run_single_experiment(args):
    graph, source, sink, seed, output_dir, filename_base, run = args
    solver = MaxFlowTabuSearchOptimized(graph, source, sink)
    result = solver.search(seed=seed)
    plot_path = os.path.join(output_dir, f"{filename_base}_run{run+1}_convergence.png")
    solver.plot_convergence(plot_path, title_suffix=f"(Run {run+1})")
    print(f"Run {run + 1}: Best={result['best_value']:.2f}, Iter={result['best_iteration']}, "
          f"Eval={result['evaluations']}, Time={result['elapsed_time']:.4f}s")
    return result


def compute_mean_convergence(runs_results, max_iterations):
    common_iterations = np.linspace(0, max_iterations, 1000)
    all_flows = []
    
    for run in runs_results:
        interp_flow = np.interp(
            common_iterations,
            run['convergence_iterations'],
            run['convergence_history'],
            right=run['convergence_history'][-1]
        )
        all_flows.append(interp_flow)
    
    mean_flow = np.mean(all_flows, axis=0)
    std_flow = np.std(all_flows, axis=0)
    
    return common_iterations, mean_flow, std_flow


def get_best_run(runs_results):
    scores = [r['best_value'] - (0.0001 * r['best_iteration']) for r in runs_results]
    best_idx = np.argmax(scores)
    return runs_results[best_idx]


def run_experiments(output_dir="results"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    instance_files = [f for f in os.listdir() if f.startswith("network_") and f.endswith(".txt")]
    results = {}
    
    for filename in sorted(instance_files):
        print(f"\n--- Processing instance: {filename} ---")
        try:
            graph, source, sink, n_nodes, n_edges = read_instance(filename)
            args_list = [(graph, source, sink, run + 42, output_dir, 
                         os.path.splitext(filename)[0], run) for run in range(10)]
            
            with Pool(max(1, cpu_count() // 2)) as p:
                runs_results = p.map(run_single_experiment, args_list)

            # Salva i dati grezzi
            #raw_data_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_raw_data.json")
            #with open(raw_data_path, 'w') as f:
            #    json.dump({
            #        'runs': runs_results,
            #        'optimal_flow': runs_results[0]['optimal_flow'],
            #        'upper_bound': min(runs_results[0]['source_out_capacity'], 
            #                         runs_results[0]['sink_in_capacity'])
            #    }, f)

            # Calcola convergenza media corretta
            max_iter = max([r['convergence_iterations'][-1] for r in runs_results])
            x_vals, mean_conv, std_conv = compute_mean_convergence(runs_results, max_iter)

            # Plot convergenza media
            plt.figure(figsize=(10, 6))
            plt.plot(x_vals, mean_conv, label='Mean Flow', color='blue')
            #plt.fill_between(x_vals, mean_conv - std_conv, mean_conv + std_conv, 
            #               alpha=0.2, color='blue', label='±1 Std Dev')
            plt.axhline(y=runs_results[0]['optimal_flow'], color='red', 
                       linestyle='--', label='Optimal (Edmonds-Karp)')
            #plt.axhline(y=min(runs_results[0]['source_out_capacity'], 
            #           runs_results[0]['sink_in_capacity']),
            #           color='green', linestyle=':', label='Upper Bound')
            plt.title(f'Mean Convergence for {filename}')
            plt.xlabel('Iterations')
            plt.ylabel('Flow Value')
            plt.grid(True)
            plt.legend()
            mean_plot_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_mean_convergence.png")
            plt.savefig(mean_plot_path, dpi=300, bbox_inches='tight')
            plt.close()

            # Plot miglior run (selezionata correttamente)
            best_run = get_best_run(runs_results)
            plt.figure(figsize=(10, 6))
            plt.plot(best_run['convergence_iterations'], best_run['convergence_history'],
                    label=f'Best Run (Flow={best_run["best_value"]:.2f})', color='blue')
            plt.axhline(y=best_run['optimal_flow'], color='red', 
                       linestyle='--', label='Optimal')
            #plt.axhline(y=min(best_run['source_out_capacity'], best_run['sink_in_capacity']),
            #           color='green', linestyle=':', label='Upper Bound')
            plt.title(f'Best Run Convergence for {filename}\n'
                     f'Final Flow: {best_run["best_value"]:.2f} | '
                     f'Optimal: {best_run["optimal_flow"]:.2f} | '
                     f'Iter: {best_run["best_iteration"]}')
            plt.xlabel('Iterations')
            plt.ylabel('Flow Value')
            plt.grid(True)
            plt.legend()
            best_plot_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_best_convergence.png")
            plt.savefig(best_plot_path, dpi=300, bbox_inches='tight')
            plt.close()

            # Statistiche finali
            all_flows = [r['best_value'] for r in runs_results]
            result = {
                'best': max(all_flows),
                'mean': np.mean(all_flows),
                'std': np.std(all_flows),
                'optimal': runs_results[0]['optimal_flow'],
                'upper_bound': min(runs_results[0]['source_out_capacity'], 
                                 runs_results[0]['sink_in_capacity']),
                'avg_iterations': np.mean([r['best_iteration'] for r in runs_results]),
                'avg_evaluations': np.mean([r['evaluations'] for r in runs_results]),
                'avg_time': np.mean([r['elapsed_time'] for r in runs_results])
            }
            results[filename] = result
            
            print(f"\nFinal statistics for {filename}:")
            print(f"Best Flow (TS): {result['best']:.2f} (Optimal: {result['optimal']:.2f})")
            print(f"Mean Flow (TS): {result['mean']:.2f} ± {result['std']:.2f}")
            print(f"Upper Bound: {result['upper_bound']:.2f}")
            print(f"Avg Iterations: {result['avg_iterations']:.0f}")
            print(f"Avg Evaluations: {result['avg_evaluations']:.0f}")
            print(f"Avg Time: {result['avg_time']:.4f}s")

        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")
    
    return results


if __name__ == "__main__":
    experiment_results = run_experiments()

In [ ]:
import random
import numpy as np
from collections import deque, defaultdict
import time
import os
import matplotlib.pyplot as plt
from typing import Dict, Tuple, List, Optional
from multiprocessing import Pool, cpu_count
import json


class MaxFlowTabuSearchOptimized:
    def __init__(self, graph: Dict[Tuple[int, int], float], source: int, sink: int,
                 tabu_tenure=20, max_iterations=20000):
        self.source = source
        self.sink = sink
        self.graph = graph
        self.edges = list(graph.keys())
        self.capacities = np.array([graph[e] for e in self.edges], dtype=np.float32)
        self.edge_to_idx = {(u, v): i for i, (u, v) in enumerate(self.edges)}
        self.source_edges = [i for i, (u, v) in enumerate(self.edges) if u == self.source]
        self.n_edges = len(self.edges)
        self.tabu_tenure = tabu_tenure
        self.max_iterations = max_iterations
        self.current_flow = np.zeros(len(self.edges), dtype=np.float32)
        self.best_flow = np.zeros(len(self.edges), dtype=np.float32)
        self.best_value = 0.0
        self.convergence_history = []
        self.convergence_iterations = []
        self.evaluations = 0
        self.best_iteration = 0
        self.start_time = time.time()
        self._precompute_adjacency()
        self.optimal_max_flow_value = self._run_edmonds_karp()
        self.source_out_capacity = sum(cap for (u, v), cap in self.graph.items() if u == self.source)
        self.sink_in_capacity = sum(cap for (u, v), cap in self.graph.items() if v == self.sink)
        self._init_with_partial_edmonds_karp()

    def _precompute_adjacency(self):
        self.adj = defaultdict(set)
        for (u, v) in self.graph:
            self.adj[u].add(v)

    def _run_edmonds_karp(self):
        residual = defaultdict(float)
        for (u, v), cap in self.graph.items():
            residual[(u, v)] = cap
            residual[(v, u)] = 0.0
        flow = 0.0
        while True:
            parent = {}
            visited = {self.source}
            queue = deque([self.source])
            found = False
            while queue and not found:
                u = queue.popleft()
                for v in self.adj[u]:
                    if residual[(u, v)] > 1e-6 and v not in visited:
                        visited.add(v)
                        parent[v] = u
                        if v == self.sink:
                            found = True
                            break
                        queue.append(v)
            if not found:
                break
            path_flow = float('inf')
            v = self.sink
            while v != self.source:
                u = parent[v]
                path_flow = min(path_flow, residual[(u, v)])
                v = u
            v = self.sink
            while v != self.source:
                u = parent[v]
                residual[(u, v)] -= path_flow
                residual[(v, u)] += path_flow
                v = u
            flow += path_flow
        return flow

    def _init_with_partial_edmonds_karp(self):
        optimal_flow = np.zeros(len(self.edges), dtype=np.float32)
        residual = defaultdict(float)
        for idx, (u, v) in enumerate(self.edges):
            residual[(u, v)] = self.capacities[idx]
            residual[(v, u)] = 0.0
        while True:
            parent = {}
            visited = {self.source}
            queue = deque([self.source])
            found = False
            while queue and not found:
                u = queue.popleft()
                for v in self.adj[u]:
                    if residual[(u, v)] > 1e-6 and v not in visited:
                        visited.add(v)
                        parent[v] = u
                        if v == self.sink:
                            found = True
                            break
                        queue.append(v)
            if not found:
                break
            path_flow = float('inf')
            v = self.sink
            while v != self.source:
                u = parent[v]
                path_flow = min(path_flow, residual[(u, v)])
                v = u
            v = self.sink
            while v != self.source:
                u = parent[v]
                edge = (u, v)
                idx = self.edge_to_idx.get(edge, -1)
                if idx >= 0:
                    residual[(u, v)] -= path_flow
                    residual[(v, u)] += path_flow
                    optimal_flow[idx] += path_flow
                v = u
        factor = random.uniform(0.7, 0.95)
        self.current_flow[:] = optimal_flow * factor
        self.best_value = self._calculate_flow_value(self.current_flow)
        self.best_flow[:] = self.current_flow
        self.convergence_history = [self.best_value]
        self.convergence_iterations = [0]

    def _calculate_flow_value(self, flow):
        self.evaluations += 1
        return np.sum(flow[self.source_edges])

    def _get_neighborhood_moves(self, current_flow):
        moves = []
        sample_size = min(len(self.edges), max(50, len(self.edges) // 40))
        sampled_edges = random.sample(range(len(self.edges)), sample_size)
        for edge_idx in sampled_edges:
            curr = current_flow[edge_idx]
            cap = self.capacities[edge_idx]
            if curr < cap - 1e-6:
                step = min(cap - curr, cap * 0.2)
                new_flow = current_flow.copy()
                new_flow[edge_idx] += step
                is_tabu = edge_idx in self.tabu_dict and self.tabu_dict[edge_idx] > 0
                moves.append((new_flow, edge_idx, step, is_tabu))
            if curr > 1e-6:
                step = min(curr, cap * 0.2)
                new_flow = current_flow.copy()
                new_flow[edge_idx] -= step
                is_tabu = edge_idx in self.tabu_dict and self.tabu_dict[edge_idx] > 0
                moves.append((new_flow, edge_idx, -step, is_tabu))
        if self.n_edges <= 1000 and len(sampled_edges) >= 2:
            edge1, edge2 = random.sample(sampled_edges, 2)
            if current_flow[edge1] > 1e-6 and current_flow[edge2] < self.capacities[edge2] - 1e-6:
                transfer = min(current_flow[edge1], self.capacities[edge2] - current_flow[edge2]) * 0.3
                new_flow = current_flow.copy()
                new_flow[edge1] -= transfer
                new_flow[edge2] += transfer
                moves.append((new_flow, (edge1, edge2), transfer, False))
        return moves

    def _diversify_solution(self, current_flow):
        reset_factor = random.uniform(0.5, 0.8)
        noise_factor = random.uniform(0.1, 0.2)
        new_flow = current_flow * (1 - reset_factor)
        num_perturb = min(len(self.edges) // 10, 10)
        indices = random.sample(range(len(self.edges)), num_perturb)
        for idx in indices:
            cap = self.capacities[idx]
            noise = random.uniform(-noise_factor, noise_factor) * cap
            new_flow[idx] = max(0, min(cap, new_flow[idx] + noise))
        return new_flow

    def plot_convergence(self, filename=None, title_suffix=""):
        plt.figure(figsize=(10, 6))
        plt.plot(self.convergence_iterations, self.convergence_history, label='Best Flow', color='blue')
        plt.axhline(y=self.optimal_max_flow_value, color='red', linestyle='--', label='Optimal (Edmonds-Karp)')
        #plt.axhline(y=min(self.source_out_capacity, self.sink_in_capacity), 
        #           color='green', linestyle=':', label='Upper Bound')
        plt.title(f'Convergence {title_suffix}')
        plt.xlabel('Iterations')
        plt.ylabel('Flow Value')
        plt.grid(True)
        plt.legend()
        if filename:
            plt.savefig(filename, dpi=300, bbox_inches='tight')
            plt.close()
        else:
            plt.show()

    def search(self, seed=None):
        if seed is not None:
            random.seed(seed)
            np.random.seed(seed)
        stagnation_counter = 0
        max_stagnation = 1000
        self.tabu_dict = defaultdict(int)
        upper_bound = min(self.source_out_capacity, self.sink_in_capacity)
        for iteration in range(self.max_iterations):
            if stagnation_counter >= max_stagnation:
                self.current_flow = self._diversify_solution(self.current_flow)
                stagnation_counter = 0
            moves = self._get_neighborhood_moves(self.current_flow)
            if not moves:
                break
            best_move = None
            best_val = -np.inf
            for move_flow, idx, delta, is_tabu in moves:
                val = self._calculate_flow_value(move_flow)
                if (not is_tabu or val > self.best_value) and val > best_val:
                    best_val = val
                    best_move = (move_flow, idx)
            if best_move is None:
                stagnation_counter += 1
                continue
            new_flow, edge_idx = best_move
            self.current_flow = new_flow
            if isinstance(edge_idx, tuple):
                for e in edge_idx:
                    self.tabu_dict[e] = self.tabu_tenure
            else:
                self.tabu_dict[edge_idx] = self.tabu_tenure
            self.tabu_dict = {k: v-1 for k, v in self.tabu_dict.items() if v > 1}
            current_val = self._calculate_flow_value(self.current_flow)
            if current_val > self.best_value:
                self.best_value = current_val
                self.best_flow[:] = self.current_flow
                self.best_iteration = iteration
                stagnation_counter = 0
            else:
                stagnation_counter += 1
            if iteration % 100 == 0 or current_val > self.convergence_history[-1]:
                self.convergence_history.append(current_val)
                self.convergence_iterations.append(iteration)
            if abs(current_val - upper_bound) < 1e-6:
                break
        elapsed_time = time.time() - self.start_time
        return {
            'convergence_history': self.convergence_history,
            'convergence_iterations': self.convergence_iterations,
            'best_value': self.best_value,
            'best_flow': self.best_flow,
            'best_iteration': self.best_iteration,
            'evaluations': self.evaluations,
            'elapsed_time': elapsed_time,
            'source_out_capacity': self.source_out_capacity,
            'sink_in_capacity': self.sink_in_capacity,
            'optimal_flow': self.optimal_max_flow_value
        }


def read_instance(filename):
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f.readlines() if line.strip()]
    n_nodes = int(lines[0])
    n_edges = int(lines[1])
    source = int(lines[2])
    sink = int(lines[3])
    graph = {}
    for i in range(4, min(4 + n_edges, len(lines))):
        parts = lines[i].split()
        if len(parts) >= 3:
            u, v, capacity = int(parts[0]), int(parts[1]), float(parts[2])
            graph[(u, v)] = capacity
    return graph, source, sink, n_nodes, n_edges


def run_single_experiment(args):
    graph, source, sink, seed, output_dir, filename_base, run = args
    solver = MaxFlowTabuSearchOptimized(graph, source, sink)
    result = solver.search(seed=seed)
    plot_path = os.path.join(output_dir, f"{filename_base}_run{run+1}_convergence.png")
    solver.plot_convergence(plot_path, title_suffix=f"(Run {run+1})")
    print(f"Run {run + 1}: Best={result['best_value']:.2f}, Iter={result['best_iteration']}, "
          f"Eval={result['evaluations']}, Time={result['elapsed_time']:.4f}s")
    return result


def compute_mean_convergence(runs_results, max_iterations):
    common_iterations = np.linspace(0, max_iterations, 1000)
    all_flows = []
    
    for run in runs_results:
        interp_flow = np.interp(
            common_iterations,
            run['convergence_iterations'],
            run['convergence_history'],
            right=run['convergence_history'][-1]
        )
        all_flows.append(interp_flow)
    
    mean_flow = np.mean(all_flows, axis=0)
    std_flow = np.std(all_flows, axis=0)
    
    return common_iterations, mean_flow, std_flow


def get_best_run(runs_results):
    scores = [r['best_value'] - (0.0001 * r['best_iteration']) for r in runs_results]
    best_idx = np.argmax(scores)
    return runs_results[best_idx]


def run_experiments(output_dir="results"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    instance_files = [f for f in os.listdir() if f.startswith("network_") and f.endswith(".txt")]
    results = {}
    
    for filename in sorted(instance_files):
        print(f"\n--- Processing instance: {filename} ---")
        try:
            graph, source, sink, n_nodes, n_edges = read_instance(filename)
            args_list = [(graph, source, sink, run + 42, output_dir, 
                         os.path.splitext(filename)[0], run) for run in range(10)]
            
            with Pool(max(1, cpu_count() // 2)) as p:
                runs_results = p.map(run_single_experiment, args_list)

            # Salva i dati grezzi
            #raw_data_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_raw_data.json")
            #with open(raw_data_path, 'w') as f:
            #    json.dump({
            #        'runs': runs_results,
            #        'optimal_flow': runs_results[0]['optimal_flow'],
            #        'upper_bound': min(runs_results[0]['source_out_capacity'], 
            #                         runs_results[0]['sink_in_capacity'])
            #    }, f)

            # Calcola convergenza media corretta
            max_iter = max([r['convergence_iterations'][-1] for r in runs_results])
            x_vals, mean_conv, std_conv = compute_mean_convergence(runs_results, max_iter)

            # Plot convergenza media
            plt.figure(figsize=(10, 6))
            plt.plot(x_vals, mean_conv, label='Mean Flow', color='blue')
            #plt.fill_between(x_vals, mean_conv - std_conv, mean_conv + std_conv, 
            #               alpha=0.2, color='blue', label='±1 Std Dev')
            plt.axhline(y=runs_results[0]['optimal_flow'], color='red', 
                       linestyle='--', label='Optimal (Edmonds-Karp)')
            #plt.axhline(y=min(runs_results[0]['source_out_capacity'], 
            #           runs_results[0]['sink_in_capacity']),
            #           color='green', linestyle=':', label='Upper Bound')
            plt.title(f'Mean Convergence for {filename}')
            plt.xlabel('Iterations')
            plt.ylabel('Flow Value')
            plt.grid(True)
            plt.legend()
            mean_plot_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_mean_convergence.png")
            plt.savefig(mean_plot_path, dpi=300, bbox_inches='tight')
            plt.close()

            # Plot miglior run (selezionata correttamente)
            best_run = get_best_run(runs_results)
            plt.figure(figsize=(10, 6))
            plt.plot(best_run['convergence_iterations'], best_run['convergence_history'],
                    label=f'Best Run (Flow={best_run["best_value"]:.2f})', color='blue')
            plt.axhline(y=best_run['optimal_flow'], color='red', 
                       linestyle='--', label='Optimal')
            #plt.axhline(y=min(best_run['source_out_capacity'], best_run['sink_in_capacity']),
            #           color='green', linestyle=':', label='Upper Bound')
            plt.title(f'Best Run Convergence for {filename}\n'
                     f'Final Flow: {best_run["best_value"]:.2f} | '
                     f'Optimal: {best_run["optimal_flow"]:.2f} | '
                     f'Iter: {best_run["best_iteration"]}')
            plt.xlabel('Iterations')
            plt.ylabel('Flow Value')
            plt.grid(True)
            plt.legend()
            best_plot_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_best_convergence.png")
            plt.savefig(best_plot_path, dpi=300, bbox_inches='tight')
            plt.close()

            # Statistiche finali
            all_flows = [r['best_value'] for r in runs_results]
            result = {
                'best': max(all_flows),
                'mean': np.mean(all_flows),
                'std': np.std(all_flows),
                'optimal': runs_results[0]['optimal_flow'],
                'upper_bound': min(runs_results[0]['source_out_capacity'], 
                                 runs_results[0]['sink_in_capacity']),
                'avg_iterations': np.mean([r['best_iteration'] for r in runs_results]),
                'avg_evaluations': np.mean([r['evaluations'] for r in runs_results]),
                'avg_time': np.mean([r['elapsed_time'] for r in runs_results])
            }
            results[filename] = result
            
            print(f"\nFinal statistics for {filename}:")
            print(f"Best Flow (TS): {result['best']:.2f} (Optimal: {result['optimal']:.2f})")
            print(f"Mean Flow (TS): {result['mean']:.2f} ± {result['std']:.2f}")
            print(f"Upper Bound: {result['upper_bound']:.2f}")
            print(f"Avg Iterations: {result['avg_iterations']:.0f}")
            print(f"Avg Evaluations: {result['avg_evaluations']:.0f}")
            print(f"Avg Time: {result['avg_time']:.4f}s")

        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")
    
    return results


if __name__ == "__main__":
    experiment_results = run_experiments()

In [ ]:
import os
import zipfile
from google.colab import files

def download_folder(folder_path):
    # Crea un nome per il file zip
    zip_name = f"{os.path.basename(folder_path)}.zip"

    # Crea un archivio zip della cartella
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files_list in os.walk(folder_path):
            for file in files_list:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, start=folder_path)
                zipf.write(file_path, arcname)

    # Scarica il file zip (versione corretta)
    from google.colab import files
    files.download(zip_name)

# Esempio di utilizzo:
download_folder('/content/results')

In [ ]:
import random
import numpy as np
from collections import defaultdict, deque
import time
import os
import matplotlib.pyplot as plt
from typing import Dict, Tuple, List, Optional
from multiprocessing import Pool, cpu_count

class MaxFlowTabuSearchCombined:
    def __init__(self, graph: Dict[Tuple[int, int], float], source: int, sink: int,
                 tabu_tenure=20, max_iterations=20000):
        self.source = source
        self.sink = sink
        self.graph = graph
        self.edges = list(graph.keys())
        self.capacities = np.array([graph[e] for e in self.edges], dtype=np.float32)
        self.edge_to_idx = {(u, v): i for i, (u, v) in enumerate(self.edges)}
        self.source_edges = [i for i, (u, v) in enumerate(self.edges) if u == self.source]
        self.n_edges = len(self.edges)
        self.tabu_tenure = tabu_tenure
        self.max_iterations = max_iterations
        self.current_flow = np.zeros(len(self.edges), dtype=np.float32)
        self.best_flow = np.zeros(len(self.edges), dtype=np.float32)
        self.best_value = 0.0
        self.convergence_history = []
        self.convergence_iterations = []
        self.evaluations = 0
        self.best_iteration = 0
        self.start_time = time.time()
        self._precompute_adjacency()
        self.optimal_max_flow_value = self._run_edmonds_karp()
        self.source_out_capacity = sum(cap for (u, v), cap in self.graph.items() if u == self.source)
        self.sink_in_capacity = sum(cap for (u, v), cap in self.graph.items() if v == self.sink)
        self._init_with_partial_edmonds_karp()

    def _precompute_adjacency(self):
        """Build adjacency lists for faster navigation."""
        self.adj = defaultdict(list)
        for (u, v) in self.graph:
            self.adj[u].append(v)

    def _run_edmonds_karp(self) -> float:
        """Edmonds-Karp implementation for optimal flow."""
        residual = defaultdict(float)
        for (u, v), cap in self.graph.items():
            residual[(u, v)] = cap
            residual[(v, u)] = 0.0
        flow = 0.0
        while True:
            parent = {}
            visited = {self.source}
            queue = deque([self.source])
            found = False
            while queue and not found:
                u = queue.popleft()
                for v in self.adj[u]:
                    if residual[(u, v)] > 1e-6 and v not in visited:
                        visited.add(v)
                        parent[v] = u
                        if v == self.sink:
                            found = True
                            break
                        queue.append(v)
            if not found:
                break
            path_flow = float('inf')
            v = self.sink
            while v != self.source:
                u = parent[v]
                path_flow = min(path_flow, residual[(u, v)])
                v = u
            v = self.sink
            while v != self.source:
                u = parent[v]
                residual[(u, v)] -= path_flow
                residual[(v, u)] += path_flow
                v = u
            flow += path_flow
        return flow

    def _init_with_partial_edmonds_karp(self):
        """Initialize with partial solution from Edmonds-Karp."""
        optimal_flow = np.zeros(len(self.edges), dtype=np.float32)
        residual = defaultdict(float)
        for idx, (u, v) in enumerate(self.edges):
            residual[(u, v)] = self.capacities[idx]
            residual[(v, u)] = 0.0
        while True:
            parent = {}
            visited = {self.source}
            queue = deque([self.source])
            found = False
            while queue and not found:
                u = queue.popleft()
                for v in self.adj[u]:
                    if residual[(u, v)] > 1e-6 and v not in visited:
                        visited.add(v)
                        parent[v] = u
                        if v == self.sink:
                            found = True
                            break
                        queue.append(v)
            if not found:
                break
            path_flow = float('inf')
            v = self.sink
            while v != self.source:
                u = parent[v]
                path_flow = min(path_flow, residual[(u, v)])
                v = u
            v = self.sink
            while v != self.source:
                u = parent[v]
                edge = (u, v)
                idx = self.edge_to_idx.get(edge, -1)
                if idx >= 0:
                    residual[(u, v)] -= path_flow
                    residual[(v, u)] += path_flow
                    optimal_flow[idx] += path_flow
                v = u
        factor = random.uniform(0.7, 0.95)
        self.current_flow[:] = optimal_flow * factor
        self.best_value = self._calculate_flow_value(self.current_flow)
        self.best_flow[:] = self.current_flow
        self.convergence_history = [self.best_value]
        self.convergence_iterations = [0]

    def _calculate_flow_value(self, flow):
        self.evaluations += 1
        return np.sum(flow[self.source_edges])

    def _get_neighborhood_moves(self, current_flow, iteration):
        moves = []
        sample_size = min(len(self.edges), max(30, len(self.edges) // 80))  # Reduced sampling
        sampled_edges = random.sample(range(len(self.edges)), sample_size)
        for edge_idx in sampled_edges:
            curr = current_flow[edge_idx]
            cap = self.capacities[edge_idx]
            if curr < cap - 1e-6:
                step = min(cap - curr, cap * 0.2)
                new_flow = current_flow.copy()
                new_flow[edge_idx] += step
                is_tabu = edge_idx in self.tabu_dict and self.tabu_dict[edge_idx] > iteration
                moves.append((new_flow, edge_idx, step, is_tabu))
            if curr > 1e-6:
                step = min(curr, cap * 0.2)
                new_flow = current_flow.copy()
                new_flow[edge_idx] -= step
                is_tabu = edge_idx in self.tabu_dict and self.tabu_dict[edge_idx] > iteration
                moves.append((new_flow, edge_idx, -step, is_tabu))
        return moves

    def _diversify_solution(self, current_flow):
        reset_factor = random.uniform(0.6, 0.8)
        noise_factor = random.uniform(0.1, 0.2)
        new_flow = current_flow * (1 - reset_factor)
        num_perturb = min(len(self.edges) // 20, 5)
        indices = random.sample(range(len(self.edges)), num_perturb)
        for idx in indices:
            cap = self.capacities[idx]
            noise = random.uniform(-noise_factor, noise_factor) * cap
            new_flow[idx] = max(0, min(cap, new_flow[idx] + noise))
        return new_flow

    def _adaptive_parameters(self, iteration, stagnation_counter):
        if stagnation_counter > 100:
            self.tabu_tenure = min(int(self.tabu_tenure * 1.2), 150)
        elif stagnation_counter < 50:
            self.tabu_tenure = max(int(self.tabu_tenure * 0.9), 10)
        if iteration % 2000 == 0 and stagnation_counter > 300:
            self.current_flow = self._diversify_solution(self.current_flow)
            return True
        return False

    def plot_convergence(self, filename=None, title_suffix=""):
        plt.figure(figsize=(12, 7))
        plt.plot(self.convergence_iterations, self.convergence_history, 'b-', label='Best Flow Value', linewidth=2)
        plt.axhline(y=self.optimal_max_flow_value, color='r', linestyle='--',
                   linewidth=1.5, label=f'Optimal (EK): {self.optimal_max_flow_value:.2f}')
        plt.scatter([self.best_iteration], [self.best_value], color='g',
                   s=100, zorder=5, label=f'Best: {self.best_value:.2f} at {self.best_iteration}')
        plt.title(f'Tabu Search Convergence {title_suffix}\n'
                 f'Nodes: {len(set(n for n, _ in self.graph))}, Edges: {len(self.graph)}', pad=20)
        plt.xlabel('Iterations', labelpad=10)
        plt.ylabel('Flow Value', labelpad=10)
        plt.grid(True, alpha=0.3)
        plt.legend(loc='lower right')
        info_text = (f"Optimal: {self.optimal_max_flow_value:.2f}\n"
                     f"Evaluations: {self.evaluations}\n"
                     f"Time: {time.time() - self.start_time:.2f}s")
        plt.gcf().text(0.15, 0.7, info_text, bbox=dict(facecolor='white', alpha=0.8))
        if filename:
            plt.savefig(filename, dpi=300, bbox_inches='tight')
            plt.close()
        else:
            plt.show()

    def search(self, seed=None):
        if seed is not None:
            random.seed(seed)
            np.random.seed(seed)

        self.tabu_dict = {}
        stagnation_counter = 0
        upper_bound = min(self.source_out_capacity, self.sink_in_capacity)
        temp_flow = np.zeros_like(self.current_flow)

        for iteration in range(self.max_iterations):
            diversified = self._adaptive_parameters(iteration, stagnation_counter)
            if diversified:
                stagnation_counter = 0

            moves = self._get_neighborhood_moves(self.current_flow, iteration)
            best_move = None
            best_val = -np.inf

            for move_flow, idx, delta, is_tabu in moves:
                val = self._calculate_flow_value(move_flow)
                if not is_tabu or val > self.best_value:
                    if val > best_val:
                        best_val = val
                        best_move = (move_flow, idx)

            if best_move is None:
                stagnation_counter += 1
                continue

            new_flow, edge_idx = best_move
            self.current_flow = new_flow

            if isinstance(edge_idx, tuple):
                for e in edge_idx:
                    self.tabu_dict[e] = iteration + self.tabu_tenure
            else:
                self.tabu_dict[edge_idx] = iteration + self.tabu_tenure

            # Update tabu list less frequently
            if iteration % 500 == 0:
                self.tabu_dict = {k: v for k, v in self.tabu_dict.items() if v > iteration}

            current_val = self._calculate_flow_value(self.current_flow)

            if current_val > self.best_value:
                self.best_value = current_val
                self.best_flow[:] = self.current_flow
                self.best_iteration = iteration
                stagnation_counter = 0
            else:
                stagnation_counter += 1

            if iteration % 100 == 0 or current_val > self.convergence_history[-1] or iteration == self.max_iterations - 1:
                self.convergence_history.append(current_val)
                self.convergence_iterations.append(iteration)

            # Early stopping based on theoretical bound
            if abs(current_val - upper_bound) < 1e-6:
                break

        elapsed_time = time.time() - self.start_time
        return {
            'best_value': self.best_value,
            'best_flow': self.best_flow.tolist(),
            'best_iteration': self.best_iteration,
            'evaluations': self.evaluations,
            'elapsed_time': elapsed_time,
            'convergence_history': self.convergence_history,
            'convergence_iterations': self.convergence_iterations,
            'optimal_flow': self.optimal_max_flow_value
        }

def read_instance(filename: str) -> Tuple[Dict[Tuple[int, int], float], int, int, int, int]:
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f.readlines() if line.strip()]
    n_nodes = int(lines[0])
    n_edges = int(lines[1])
    source = int(lines[2])
    sink = int(lines[3])
    graph = {}
    for i in range(4, min(4 + n_edges, len(lines))):
        parts = lines[i].split()
        if len(parts) >= 3:
            u, v, capacity = int(parts[0]), int(parts[1]), float(parts[2])
            graph[(u, v)] = capacity
    return graph, source, sink, n_nodes, n_edges

def run_single_experiment(args):
    graph, source, sink, seed, output_dir, filename_base, run = args
    solver = MaxFlowTabuSearchCombined(graph, source, sink)
    result = solver.search(seed=seed)
    plot_path = os.path.join(output_dir, f"{filename_base}_run{run+1}_convergence.png")
    solver.plot_convergence(plot_path, title_suffix=f"(Run {run+1})")
    print(f"Run {run + 1}: Best={result['best_value']:.2f}, Iter={result['best_iteration']}, "
          f"Eval={result['evaluations']}, Time={result['elapsed_time']:.4f}s")
    return result

def compute_mean_convergence(runs_results, max_iterations):
    common_iterations = np.linspace(0, max_iterations, 1000)
    all_flows = []
    
    for run in runs_results:
        interp_flow = np.interp(
            common_iterations,
            run['convergence_iterations'],
            run['convergence_history'],
            right=run['convergence_history'][-1]
        )
        all_flows.append(interp_flow)
    
    mean_flow = np.mean(all_flows, axis=0)
    std_flow = np.std(all_flows, axis=0)
    
    return common_iterations, mean_flow, std_flow


def get_best_run(runs_results):
    scores = [r['best_value'] - (0.0001 * r['best_iteration']) for r in runs_results]
    best_idx = np.argmax(scores)
    return runs_results[best_idx]

def plot_average_convergence(results, filename, optimal_value):
    plt.figure(figsize=(12, 7))
    max_iter = max(len(r['convergence_iterations']) for r in results)
    x_vals, mean_conv, std_conv = compute_mean_convergence(results, max_iter)
    plt.plot(x_vals, mean_conv, 'b-', linewidth=2, label='Average Flow')
    plt.fill_between(x_vals, mean_conv - std_conv, mean_conv + std_conv, alpha=0.2)
    plt.axhline(y=optimal_value, color='r', linestyle='--', linewidth=1.5, label=f'Optimal (EK): {optimal_value:.2f}')
    plt.title(f'Average Convergence for {os.path.basename(filename)}\n{len(results)} runs', pad=20)
    plt.xlabel('Iterations', labelpad=10)
    plt.ylabel('Flow Value', labelpad=10)
    plt.grid(True, alpha=0.3)
    plt.legend(loc='lower right')
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()

def run_experiments(output_dir="results"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    instance_files = [f for f in os.listdir() if f.startswith("network_") and f.endswith(".txt")]
    results = {}
    for filename in sorted(instance_files):
        print(f"\n--- Processing instance: {filename} ---")
        try:
            graph, source, sink, n_nodes, n_edges = read_instance(filename)
            args_list = [(graph, source, sink, run + 42, output_dir,
                         os.path.splitext(os.path.basename(filename))[0], run) for run in range(10)]
            with Pool(max(1, cpu_count() // 2)) as p:
                runs_results = p.map(run_single_experiment, args_list)
            #avg_plot_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_average_convergence.png")
            #plot_average_convergence(runs_results, avg_plot_path, runs_results[0]['optimal_flow'])
            
            # Calcola convergenza media corretta
            max_iter = max([r['convergence_iterations'][-1] for r in runs_results])
            x_vals, mean_conv, std_conv = compute_mean_convergence(runs_results, max_iter)

            # Plot convergenza media
            plt.figure(figsize=(10, 6))
            plt.plot(x_vals, mean_conv, label='Mean Flow', color='blue')
            #plt.fill_between(x_vals, mean_conv - std_conv, mean_conv + std_conv, 
            #               alpha=0.2, color='blue', label='±1 Std Dev')
            plt.axhline(y=runs_results[0]['optimal_flow'], color='red', 
                       linestyle='--', label='Optimal (Edmonds-Karp)')
            #plt.axhline(y=min(runs_results[0]['source_out_capacity'], 
            #           runs_results[0]['sink_in_capacity']),
            #           color='green', linestyle=':', label='Upper Bound')
            plt.title(f'Mean Convergence for {filename}')
            plt.xlabel('Iterations')
            plt.ylabel('Flow Value')
            plt.grid(True)
            plt.legend()
            mean_plot_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_mean_convergence.png")
            plt.savefig(mean_plot_path, dpi=300, bbox_inches='tight')
            plt.close()

            # Plot miglior run (selezionata correttamente)
            best_run = get_best_run(runs_results)
            plt.figure(figsize=(10, 6))
            plt.plot(best_run['convergence_iterations'], best_run['convergence_history'],
                    label=f'Best Run (Flow={best_run["best_value"]:.2f})', color='blue')
            plt.axhline(y=best_run['optimal_flow'], color='red', 
                       linestyle='--', label='Optimal')
            #plt.axhline(y=min(best_run['source_out_capacity'], best_run['sink_in_capacity']),
            #           color='green', linestyle=':', label='Upper Bound')
            plt.title(f'Best Run Convergence for {filename}\n'
                     f'Final Flow: {best_run["best_value"]:.2f} | '
                     f'Optimal: {best_run["optimal_flow"]:.2f} | '
                     f'Iter: {best_run["best_iteration"]}')
            plt.xlabel('Iterations')
            plt.ylabel('Flow Value')
            plt.grid(True)
            plt.legend()
            best_plot_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_best_convergence.png")
            plt.savefig(best_plot_path, dpi=300, bbox_inches='tight')
            plt.close()
            
            all_flows = [r['best_value'] for r in runs_results]
            result = {
                'best': max(all_flows),
                'mean': np.mean(all_flows),
                'std': np.std(all_flows),
                'avg_iterations': np.mean([r['best_iteration'] for r in runs_results]),
                'avg_evaluations': np.mean([r['evaluations'] for r in runs_results]),
                'avg_time': np.mean([r['elapsed_time'] for r in runs_results]),
                'optimal_flow': runs_results[0]['optimal_flow']
            }
            results[filename] = result
            print(f"\nFinal statistics for {filename}:")
            print(f"Best Flow (TS): {result['best']:.2f} (Optimal: {result['optimal_flow']:.2f})")
            print(f"Mean Flow (TS): {result['mean']:.2f} ± {result['std']:.2f}")
            print(f"Avg Iterations: {result['avg_iterations']:.0f}")
            print(f"Avg Evaluations: {result['avg_evaluations']:.0f}")
            print(f"Avg Time: {result['avg_time']:.4f}s")
        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")
    return results

if __name__ == "__main__":
    experiment_results = run_experiments()

In [ ]:
import random
import numpy as np
from collections import defaultdict, deque
import time
import os
import matplotlib.pyplot as plt
from typing import Dict, Tuple, List, Optional, Literal
from multiprocessing import Pool, cpu_count

class MaxFlowTabuSearch:
    def __init__(self, graph: Dict[Tuple[int, int], float], source: int, sink: int,
                 tabu_tenure=20, max_iterations=20000,
                 initialization: Literal['random', 'ek_partial', 'greedy'] = 'ek_partial'):
        self.source = source
        self.sink = sink
        self.graph = graph
        self.edges = list(graph.keys())
        self.capacities = np.array([graph[e] for e in self.edges], dtype=np.float32)
        self.edge_to_idx = {(u, v): i for i, (u, v) in enumerate(self.edges)}
        self.source_edges = [i for i, (u, v) in enumerate(self.edges) if u == self.source]
        self.n_edges = len(self.edges)
        self.tabu_tenure = tabu_tenure
        self.max_iterations = max_iterations
        self.initialization = initialization
        self.current_flow = np.zeros(len(self.edges), dtype=np.float32)
        self.best_flow = np.zeros(len(self.edges), dtype=np.float32)
        self.best_value = 0.0
        self.convergence_history = []
        self.convergence_iterations = []
        self.evaluations = 0
        self.best_iteration = 0
        self.start_time = time.time()
        self._precompute_adjacency()
        self.optimal_max_flow_value = self._run_edmonds_karp()
        self.source_out_capacity = sum(cap for (u, v), cap in self.graph.items() if u == self.source)
        self.sink_in_capacity = sum(cap for (u, v), cap in self.graph.items() if v == self.sink)
        self._initialize_flow()
        self.tabu_dict = {}

    def _precompute_adjacency(self):
        self.adj = defaultdict(list)
        for (u, v) in self.graph:
            self.adj[u].append(v)

    def _run_edmonds_karp(self) -> float:
        residual = defaultdict(float)
        for (u, v), cap in self.graph.items():
            residual[(u, v)] = cap
            residual[(v, u)] = 0.0
        flow = 0.0
        while True:
            parent = {}
            visited = {self.source}
            queue = deque([self.source])
            found = False
            while queue and not found:
                u = queue.popleft()
                for v in self.adj[u]:
                    if residual[(u, v)] > 1e-6 and v not in visited:
                        visited.add(v)
                        parent[v] = u
                        if v == self.sink:
                            found = True
                            break
                        queue.append(v)
            if not found:
                break
            path_flow = float('inf')
            v = self.sink
            while v != self.source:
                u = parent[v]
                path_flow = min(path_flow, residual[(u, v)])
                v = u
            v = self.sink
            while v != self.source:
                u = parent[v]
                residual[(u, v)] -= path_flow
                residual[(v, u)] += path_flow
                v = u
            flow += path_flow
        return flow

    def _initialize_flow(self):
        if self.initialization == 'random':
            self._random_initialization()
        elif self.initialization == 'ek_partial':
            self._ek_partial_initialization()
        elif self.initialization == 'ek_full':
            self._ek_full_initialization()
        elif self.initialization == 'greedy':
            self._greedy_initialization()
        
        self.best_value = self._calculate_flow_value(self.current_flow)
        self.best_flow = self.current_flow.copy()
        self.convergence_history = [self.best_value]
        self.convergence_iterations = [0]

    def _random_initialization(self):
        """Initialize with random flows respecting capacities"""
        self.current_flow = np.array([random.uniform(0, cap) for cap in self.capacities], dtype=np.float32)
        
    def _ek_partial_initialization(self):
        """Initialize with a fraction (70-95%) of the Edmonds-Karp solution"""
        optimal_flow = np.zeros(len(self.edges), dtype=np.float32)
        residual = defaultdict(float)
        for idx, (u, v) in enumerate(self.edges):
            residual[(u, v)] = self.capacities[idx]
            residual[(v, u)] = 0.0
        
        while True:
            parent = {}
            visited = {self.source}
            queue = deque([self.source])
            found = False
            while queue and not found:
                u = queue.popleft()
                for v in self.adj[u]:
                    if residual[(u, v)] > 1e-6 and v not in visited:
                        visited.add(v)
                        parent[v] = u
                        if v == self.sink:
                            found = True
                            break
                        queue.append(v)
            if not found:
                break
            path_flow = float('inf')
            v = self.sink
            while v != self.source:
                u = parent[v]
                path_flow = min(path_flow, residual[(u, v)])
                v = u
            v = self.sink
            while v != self.source:
                u = parent[v]
                residual[(u, v)] -= path_flow
                residual[(v, u)] += path_flow
                optimal_flow[self.edge_to_idx[(u, v)]] += path_flow
                v = u
        
        factor = random.uniform(0.7, 0.95)
        self.current_flow = optimal_flow * factor
                 
    def _greedy_initialization(self):
        """Initialize by greedily pushing flow from source to sink along paths with max residual capacity"""
        self.current_flow = np.zeros(len(self.edges), dtype=np.float32)
        residual = self.capacities.copy()
        
        for _ in range(10):  # Perform a limited number of greedy pushes
            path = self._find_augmenting_path_greedy(residual)
            if not path:
                break
            min_residual = min(residual[edge_idx] for edge_idx in path)
            for edge_idx in path:
                self.current_flow[edge_idx] += min_residual
                residual[edge_idx] -= min_residual

    def _find_augmenting_path_greedy(self, residual):
        """Find a path with maximum residual capacity using modified Dijkstra's algorithm"""
        parent = {}
        capacity = {}
        visited = set()
        queue = [self.source]
        visited.add(self.source)
        capacity[self.source] = float('inf')
        
        while queue:
            u = max(queue, key=lambda x: capacity[x])
            queue.remove(u)
            
            for v in self.adj[u]:
                edge_idx = self.edge_to_idx[(u, v)]
                if residual[edge_idx] > 1e-6 and v not in visited:
                    visited.add(v)
                    parent[v] = u
                    capacity[v] = min(capacity[u], residual[edge_idx])
                    if v == self.sink:
                        path = []
                        node = self.sink
                        while node != self.source:
                            edge_idx = self.edge_to_idx[(parent[node], node)]
                            path.append(edge_idx)
                            node = parent[node]
                        return path[::-1]
                    queue.append(v)
        return None

    def _calculate_flow_value(self, flow):
        self.evaluations += 1
        return np.sum(flow[self.source_edges])

    def _find_augmenting_path(self, flow):
        parent = {}
        visited = set()
        queue = deque([self.source])
        visited.add(self.source)
        found = False
        while queue and not found:
            u = queue.popleft()
            for v in self.adj[u]:
                residual = self.graph[(u, v)] - flow[self.edge_to_idx[(u, v)]]
                if residual > 1e-6 and v not in visited:
                    visited.add(v)
                    parent[v] = u
                    if v == self.sink:
                        found = True
                        break
                    queue.append(v)
        if not found:
            return None
        path = []
        v = self.sink
        while v != self.source:
            u = parent[v]
            path.append((self.edge_to_idx[(u, v)], True))
            v = u
        return path[::-1]

    def _generate_edge_exchange_moves(self, current_flow, n_exchanges=5):
        moves = []
        saturated = [i for i in range(self.n_edges) 
                    if current_flow[i] >= self.capacities[i] - 1e-6]
        unsaturated = [i for i in range(self.n_edges) 
                      if current_flow[i] < self.capacities[i] - 1e-6]
        for _ in range(min(n_exchanges, len(saturated), len(unsaturated))):
            src, dest = random.choice(saturated), random.choice(unsaturated)
            delta = min(current_flow[src], self.capacities[dest] - current_flow[dest])
            if delta > 1e-6:
                new_flow = current_flow.copy()
                new_flow[src] -= delta
                new_flow[dest] += delta
                moves.append((new_flow, (src, dest), delta, False))
        return moves

    def _get_neighborhood_moves(self, current_flow, iteration):
        moves = []
        # Basic moves (increment/decrement)
        sample_size = min(30, self.n_edges // 80)
        for edge_idx in random.sample(range(self.n_edges), sample_size):
            curr = current_flow[edge_idx]
            cap = self.capacities[edge_idx]
            if curr < cap - 1e-6:
                step = min(cap - curr, cap * 0.2)
                new_flow = current_flow.copy()
                new_flow[edge_idx] += step
                is_tabu = edge_idx in self.tabu_dict and self.tabu_dict[edge_idx] > iteration
                moves.append((new_flow, edge_idx, step, is_tabu))
            if curr > 1e-6:
                step = min(curr, cap * 0.2)
                new_flow = current_flow.copy()
                new_flow[edge_idx] -= step
                is_tabu = edge_idx in self.tabu_dict and self.tabu_dict[edge_idx] > iteration
                moves.append((new_flow, edge_idx, -step, is_tabu))
        
        # Edge exchange (every 20 iterations)
        if iteration % 20 == 0:
            moves.extend(self._generate_edge_exchange_moves(current_flow))
        
        # Path augmentation (every 100 iterations or if stagnating)
        if iteration % 100 == 0:
            path = self._find_augmenting_path(current_flow)
            if path:
                min_residual = min(
                    self.capacities[edge_idx] - current_flow[edge_idx] if is_forward 
                    else current_flow[edge_idx]
                    for edge_idx, is_forward in path
                )
                if min_residual > 1e-6:
                    new_flow = current_flow.copy()
                    for edge_idx, is_forward in path:
                        if is_forward:
                            new_flow[edge_idx] += min_residual
                        else:
                            new_flow[edge_idx] -= min_residual
                    moves.append((new_flow, tuple(edge_idx for edge_idx, _ in path), min_residual, False))
        return moves

    def _diversify_solution(self, current_flow):
        reset_factor = random.uniform(0.6, 0.8)
        noise_factor = random.uniform(0.1, 0.2)
        new_flow = current_flow * (1 - reset_factor)
        num_perturb = min(len(self.edges) // 20, 5)
        indices = random.sample(range(len(self.edges)), num_perturb)
        for idx in indices:
            cap = self.capacities[idx]
            noise = random.uniform(-noise_factor, noise_factor) * cap
            new_flow[idx] = max(0, min(cap, new_flow[idx] + noise))
        return new_flow

    def _adaptive_parameters(self, iteration, stagnation_counter):
        if stagnation_counter > 100:
            self.tabu_tenure = min(int(self.tabu_tenure * 1.2), 150)
        elif stagnation_counter < 50:
            self.tabu_tenure = max(int(self.tabu_tenure * 0.9), 10)
        if iteration % 2000 == 0 and stagnation_counter > 300:
            self.current_flow = self._diversify_solution(self.current_flow)
            return True
        return False

    def plot_convergence(self, filename=None, title_suffix=""):
        plt.figure(figsize=(12, 7))
        plt.plot(self.convergence_iterations, self.convergence_history, 'b-', label='Best Flow Value', linewidth=2)
        plt.axhline(y=self.optimal_max_flow_value, color='r', linestyle='--',
                   linewidth=1.5, label=f'Optimal (EK): {self.optimal_max_flow_value:.2f}')
        plt.scatter([self.best_iteration], [self.best_value], color='g',
                   s=100, zorder=5, label=f'Best: {self.best_value:.2f} at {self.best_iteration}')
        plt.title(f'Tabu Search Convergence {title_suffix}\n'
                 f'Nodes: {len(set(n for n, _ in self.graph))}, Edges: {len(self.graph)}', pad=20)
        plt.xlabel('Iterations', labelpad=10)
        plt.ylabel('Flow Value', labelpad=10)
        plt.grid(True, alpha=0.3)
        plt.legend(loc='lower right')
        info_text = (f"Optimal: {self.optimal_max_flow_value:.2f}\n"
                     f"Evaluations: {self.evaluations}\n"
                     f"Time: {time.time() - self.start_time:.2f}s")
        plt.gcf().text(0.15, 0.7, info_text, bbox=dict(facecolor='white', alpha=0.8))
        if filename:
            plt.savefig(filename, dpi=300, bbox_inches='tight')
            plt.close()
        else:
            plt.show()

    def search(self, seed=None):
        if seed is not None:
            random.seed(seed)
            np.random.seed(seed)

        self.tabu_dict = {}
        stagnation_counter = 0
        upper_bound = min(self.source_out_capacity, self.sink_in_capacity)

        for iteration in range(self.max_iterations):
            diversified = self._adaptive_parameters(iteration, stagnation_counter)
            if diversified:
                stagnation_counter = 0

            moves = self._get_neighborhood_moves(self.current_flow, iteration)
            best_move = None
            best_val = -np.inf

            for move_flow, idx, delta, is_tabu in moves:
                if not is_tabu or self._calculate_flow_value(move_flow) > self.best_value:
                    val = self._calculate_flow_value(move_flow)
                    if val > best_val:
                        best_val = val
                        best_move = (move_flow, idx)

            if best_move is None:
                stagnation_counter += 1
                continue

            new_flow, edge_idx = best_move
            self.current_flow = new_flow

            if isinstance(edge_idx, tuple):
                for e in edge_idx:
                    self.tabu_dict[e] = iteration + self.tabu_tenure
            else:
                self.tabu_dict[edge_idx] = iteration + self.tabu_tenure

            if iteration % 500 == 0:
                self.tabu_dict = {k: v for k, v in self.tabu_dict.items() if v > iteration}

            current_val = self._calculate_flow_value(self.current_flow)

            if current_val > self.best_value:
                self.best_value = current_val
                self.best_flow = self.current_flow.copy()
                self.best_iteration = iteration
                stagnation_counter = 0
            else:
                stagnation_counter += 1

            if iteration % 100 == 0 or current_val > self.convergence_history[-1]:
                self.convergence_history.append(self.best_value)
                self.convergence_iterations.append(iteration)

            if abs(current_val - upper_bound) < 1e-6:
                break

        elapsed_time = time.time() - self.start_time
        return {
            'best_value': self.best_value,
            'best_flow': self.best_flow.tolist(),
            'best_iteration': self.best_iteration,
            'evaluations': self.evaluations,
            'elapsed_time': elapsed_time,
            'convergence_history': self.convergence_history,
            'convergence_iterations': self.convergence_iterations,
            'optimal_flow': self.optimal_max_flow_value
        }

# Resto del codice (read_instance, run_single_experiment, compute_mean_convergence, ...) rimane identico
def read_instance(filename: str) -> Tuple[Dict[Tuple[int, int], float], int, int, int, int]:
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f.readlines() if line.strip()]
    n_nodes = int(lines[0])
    n_edges = int(lines[1])
    source = int(lines[2])
    sink = int(lines[3])
    graph = {}
    for i in range(4, min(4 + n_edges, len(lines))):
        parts = lines[i].split()
        if len(parts) >= 3:
            u, v, capacity = int(parts[0]), int(parts[1]), float(parts[2])
            graph[(u, v)] = capacity
    return graph, source, sink, n_nodes, n_edges

def run_single_experiment(args):
    graph, source, sink, seed, output_dir, filename_base, run = args
    solver = MaxFlowTabuSearch(graph, source, sink, initialization="random")
    result = solver.search(seed=seed)
    plot_path = os.path.join(output_dir, f"{filename_base}_run{run+1}_convergence.png")
    solver.plot_convergence(plot_path, title_suffix=f"(Run {run+1})")
    print(f"Run {run + 1}: Best={result['best_value']:.2f}, Iter={result['best_iteration']}, "
          f"Eval={result['evaluations']}, Time={result['elapsed_time']:.4f}s")
    return result

def compute_mean_convergence(runs_results, max_iterations):
    common_iterations = np.linspace(0, max_iterations, 1000)
    all_flows = []

    for run in runs_results:
        interp_flow = np.interp(
            common_iterations,
            run['convergence_iterations'],
            run['convergence_history'],
            right=run['convergence_history'][-1]
        )
        all_flows.append(interp_flow)

    mean_flow = np.mean(all_flows, axis=0)
    std_flow = np.std(all_flows, axis=0)

    return common_iterations, mean_flow, std_flow


def get_best_run(runs_results):
    scores = [r['best_value'] - (0.0001 * r['best_iteration']) for r in runs_results]
    best_idx = np.argmax(scores)
    return runs_results[best_idx]

def plot_average_convergence(results, filename, optimal_value):
    plt.figure(figsize=(12, 7))
    max_iter = max(len(r['convergence_iterations']) for r in results)
    x_vals, mean_conv, std_conv = compute_mean_convergence(results, max_iter)
    plt.plot(x_vals, mean_conv, 'b-', linewidth=2, label='Average Flow')
    plt.fill_between(x_vals, mean_conv - std_conv, mean_conv + std_conv, alpha=0.2)
    plt.axhline(y=optimal_value, color='r', linestyle='--', linewidth=1.5, label=f'Optimal (EK): {optimal_value:.2f}')
    plt.title(f'Average Convergence for {os.path.basename(filename)}\n{len(results)} runs', pad=20)
    plt.xlabel('Iterations', labelpad=10)
    plt.ylabel('Flow Value', labelpad=10)
    plt.grid(True, alpha=0.3)
    plt.legend(loc='lower right')
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()

def run_experiments(output_dir="results"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    instance_files = [f for f in os.listdir() if f.startswith("network_") and f.endswith(".txt")]
    results = {}
    for filename in sorted(instance_files):
        print(f"\n--- Processing instance: {filename} ---")
        try:
            graph, source, sink, n_nodes, n_edges = read_instance(filename)
            args_list = [(graph, source, sink, run + 42, output_dir,
                         os.path.splitext(os.path.basename(filename))[0], run) for run in range(10)]
            with Pool(max(1, cpu_count() // 2)) as p:
                runs_results = p.map(run_single_experiment, args_list)
            #avg_plot_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_average_convergence.png")
            #plot_average_convergence(runs_results, avg_plot_path, runs_results[0]['optimal_flow'])

            # Calcola convergenza media corretta
            max_iter = max([r['convergence_iterations'][-1] for r in runs_results])
            x_vals, mean_conv, std_conv = compute_mean_convergence(runs_results, max_iter)

            # Plot convergenza media
            plt.figure(figsize=(10, 6))
            plt.plot(x_vals, mean_conv, label='Mean Flow', color='blue')
            #plt.fill_between(x_vals, mean_conv - std_conv, mean_conv + std_conv,
            #               alpha=0.2, color='blue', label='±1 Std Dev')
            plt.axhline(y=runs_results[0]['optimal_flow'], color='red',
                       linestyle='--', label='Optimal (Edmonds-Karp)')
            #plt.axhline(y=min(runs_results[0]['source_out_capacity'],
            #           runs_results[0]['sink_in_capacity']),
            #           color='green', linestyle=':', label='Upper Bound')
            plt.title(f'Mean Convergence for {filename}')
            plt.xlabel('Iterations')
            plt.ylabel('Flow Value')
            plt.grid(True)
            plt.legend()
            mean_plot_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_mean_convergence.png")
            plt.savefig(mean_plot_path, dpi=300, bbox_inches='tight')
            plt.close()

            # Plot miglior run (selezionata correttamente)
            best_run = get_best_run(runs_results)
            plt.figure(figsize=(10, 6))
            plt.plot(best_run['convergence_iterations'], best_run['convergence_history'],
                    label=f'Best Run (Flow={best_run["best_value"]:.2f})', color='blue')
            plt.axhline(y=best_run['optimal_flow'], color='red',
                       linestyle='--', label='Optimal')
            #plt.axhline(y=min(best_run['source_out_capacity'], best_run['sink_in_capacity']),
            #           color='green', linestyle=':', label='Upper Bound')
            plt.title(f'Best Run Convergence for {filename}\n'
                     f'Final Flow: {best_run["best_value"]:.2f} | '
                     f'Optimal: {best_run["optimal_flow"]:.2f} | '
                     f'Iter: {best_run["best_iteration"]}')
            plt.xlabel('Iterations')
            plt.ylabel('Flow Value')
            plt.grid(True)
            plt.legend()
            best_plot_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_best_convergence.png")
            plt.savefig(best_plot_path, dpi=300, bbox_inches='tight')
            plt.close()

            all_flows = [r['best_value'] for r in runs_results]
            result = {
                'best': max(all_flows),
                'mean': np.mean(all_flows),
                'std': np.std(all_flows),
                'avg_iterations': np.mean([r['best_iteration'] for r in runs_results]),
                'avg_evaluations': np.mean([r['evaluations'] for r in runs_results]),
                'avg_time': np.mean([r['elapsed_time'] for r in runs_results]),
                'optimal_flow': runs_results[0]['optimal_flow']
            }
            results[filename] = result
            print(f"\nFinal statistics for {filename}:")
            print(f"Best Flow (TS): {result['best']:.2f} (Optimal: {result['optimal_flow']:.2f})")
            print(f"Mean Flow (TS): {result['mean']:.2f} ± {result['std']:.2f}")
            print(f"Avg Iterations: {result['avg_iterations']:.0f}")
            print(f"Avg Evaluations: {result['avg_evaluations']:.0f}")
            print(f"Avg Time: {result['avg_time']:.4f}s")
        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")
    return results
                       
if __name__ == "__main__":
    experiment_results = run_experiments()